# Extract time series of GLORYS12V1 data at OOI mooring locations
Created by Ivan Lima on Tue May 10 2022 08:51:24 -0400

In [2]:
%matplotlib inline
import xarray as xr
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ccsm_utils as ccsm
import os, datetime, glob
from tqdm import notebook
print('Last updated on {}'.format(datetime.datetime.now().ctime()))

Last updated on Tue May 10 11:45:41 2022


In [3]:
sns.set_theme(context='paper', style='ticks', palette='tab10', rc={'figure.dpi':100, 'figure.figsize':[5, 5], 'axes.grid':True})

## Read GLORYS12V1 data

In [4]:
filelist = glob.glob('/bali/data/ilima/GLORYS12V1/daily/BGC/GLORYS12V1_NW_Atlantic_201?-??_BGC.nc')
filelist.sort()
ds = xr.open_mfdataset(filelist)
ds.data_vars

Data variables:
    Temperature           (time, depth, latitude, longitude) float32 dask.array<chunksize=(31, 50, 145, 169), meta=np.ndarray>
    Salinity              (time, depth, latitude, longitude) float32 dask.array<chunksize=(31, 50, 145, 169), meta=np.ndarray>
    DIC                   (time, depth, latitude, longitude) float32 dask.array<chunksize=(31, 50, 145, 169), meta=np.ndarray>
    TA                    (time, depth, latitude, longitude) float32 dask.array<chunksize=(31, 50, 145, 169), meta=np.ndarray>
    pH                    (time, depth, latitude, longitude) float64 dask.array<chunksize=(31, 50, 145, 169), meta=np.ndarray>
    saturation_calcite    (time, depth, latitude, longitude) float64 dask.array<chunksize=(31, 50, 145, 169), meta=np.ndarray>
    saturation_aragonite  (time, depth, latitude, longitude) float64 dask.array<chunksize=(31, 50, 145, 169), meta=np.ndarray>

## Extract time series at OOI moorings

In [7]:
## OOI mooring positions
# OSSM: 39.9371 N, 70.887 W
# ISSM: 40.3619 N, 70.8783 W
# CNSM: 40.1334 N, 70.7785 W
OOI_moorings = {'OSSM': (-70.8870, 39.9371),
                'ISSM': (-70.8783, 40.3619),
                'CNSM': (-70.7785, 40.1334)}

tlon, tlat = np.meshgrid(ds.longitude.values, ds.latitude.values)
now = datetime.datetime.now().ctime()

for loc in OOI_moorings:
    print('extracting {}'.format(loc))
    da_dict = {}
    for vname in notebook.tqdm(ds.data_vars):
        ts = ccsm.extract_loc_vec(*OOI_moorings[loc], tlon, tlat, ds[vname].values)
        ts = np.ma.compress_cols(ts) # remove masked values (bottom)
        zz = ts.shape[1]             # index of bottom grid cell
        da_dict[vname] = xr.DataArray(ts, coords=[ds.coords['time'], ds.coords['depth'][:zz]], dims=['time', 'depth'], name=vname)

    attrs={'contents':'Time series of GLORYS12V1 output at OOI-{} mooring'.format(loc),
           'history':'Created by Ivan Lima <ilima@whoi.edu> on {}'.format(now)}
    ds_out = xr.Dataset(da_dict)
    ds_out.attrs.update(attrs)
    for vname in ds.data_vars:
            ds_out[vname].attrs.update(ds[vname].attrs)

    ds_out.to_netcdf('data/OOI-{}_time_series.nc'.format(loc), mode='w', unlimited_dims=['time'])

extracting OSSM


  0%|          | 0/7 [00:00<?, ?it/s]

extracting ISSM


  0%|          | 0/7 [00:00<?, ?it/s]

extracting CNSM


  0%|          | 0/7 [00:00<?, ?it/s]

---

In [ ]:
fig, axs = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(15, 10))
fig.subplots_adjust(hspace=0.125)
for ax, vname in zip(axs.ravel(), ['Temperature','Salinity','DIC','TA','pH','saturation_aragonite']):
    _ = ds_out[vname].plot(x='time', y='depth', ax=ax)
    ax.invert_yaxis()
    _ = ax.set_title('OOI')

# fig.savefig('ooi.png', dpi=300, bbox_inches='tight')